Code to extract the coordinates of the keypoints from the image and writing it to the csv file


In [2]:
from ultralytics import YOLO
import cv2
import pandas as pd

In [3]:
image_paths = [
'../data/amrit_basic.jpg',
 '../data/amrit_flex.jpg',
 '../data/amrit_leg_stretch.jpg',
 '../data/amrit_lunge.jpg',
 '../data/amrit_pray.jpg',
 '../data/amrit_squat.jpg',
 '../data/amrit_surya_four.jpg',
 '../data/amrit_surya_three.jpg',
 '../data/amrit_t_pose.jpg',
 '../data/amrit_zigzag.jpg',
 '../data/pavan_dance.jpg',
 '../data/pavan_flamingo.jpg',
 '../data/pavan_full_stop.jpg',
 '../data/pavan_jesus.jpg',
 '../data/pavan_karate_kick.jpg',
 '../data/pavan_left_stretch.jpg',
 '../data/pavan_neck_stretch.jpg',
 '../data/pavan_surya_two.jpg',
 '../data/pavan_t_pose_variant.jpg',
 '../data/pavan_wicketkeeper.jpg'
 ]

In [4]:
model = YOLO('../models/yolov8n-pose.pt')

Extracting the Co-ordinates


In [4]:
all_dfs = []

for img in image_paths:
    read_img = cv2.imread(img, 1)
    img = cv2.resize(read_img, (600, 800))

    # Predict keypoints on an image
    results = model(img)
    
    # Access keypoints
    keypoints = results[0].keypoints
    tensor = keypoints.data.cpu().numpy()

    # Calculate the confidence sum for each detected person
    confidence_sums = tensor[:, :, 2].sum(axis=1)

    # Find the index of the person with the highest confidence sum
    best_person_idx = confidence_sums.argmax()

    # Extract keypoints for the most confident person
    best_person_keypoints = tensor[best_person_idx]

    # Convert to DataFrame
    df_img = pd.DataFrame(best_person_keypoints, columns=['X', 'Y', 'Confidence'])

    all_dfs.append(df_img)

# Combine all DataFrames into one if needed
final_df = pd.concat(all_dfs, ignore_index=True)

final_df


0: 640x480 1 person, 40.6ms
Speed: 7.4ms preprocess, 40.6ms inference, 1.3ms postprocess per image at shape (1, 3, 640, 480)

0: 640x480 1 person, 40.1ms
Speed: 3.5ms preprocess, 40.1ms inference, 2.2ms postprocess per image at shape (1, 3, 640, 480)

0: 640x480 2 persons, 38.6ms
Speed: 4.8ms preprocess, 38.6ms inference, 2.6ms postprocess per image at shape (1, 3, 640, 480)

0: 640x480 2 persons, 23.8ms
Speed: 1.8ms preprocess, 23.8ms inference, 1.7ms postprocess per image at shape (1, 3, 640, 480)

0: 640x480 2 persons, 18.9ms
Speed: 3.7ms preprocess, 18.9ms inference, 4.6ms postprocess per image at shape (1, 3, 640, 480)

0: 640x480 2 persons, 21.0ms
Speed: 4.7ms preprocess, 21.0ms inference, 2.5ms postprocess per image at shape (1, 3, 640, 480)

0: 640x480 1 person, 19.8ms
Speed: 3.5ms preprocess, 19.8ms inference, 3.7ms postprocess per image at shape (1, 3, 640, 480)

0: 640x480 1 person, 21.3ms
Speed: 6.1ms preprocess, 21.3ms inference, 0.0ms postprocess per image at shape (1, 3

,X,Y,Confidence
0,286.169617,236.163864,0.994265
1,300.082977,224.264435,0.978486
2,273.275116,224.945160,0.975879
3,321.056641,233.825043,0.864691
4,256.760956,235.625854,0.805083
...,...,...,...
335,267.109192,485.122833,0.999483
336,375.120483,574.618591,0.996073
337,233.417786,575.853760,0.997416
338,392.337799,652.131104,0.968906


Writing to csv


In [5]:
final_df.to_csv('yolov8n.csv')